In [ ]:
!pip install selenium
!pip install webdriver_manager

In [ ]:
!sudo apt update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install
!google-chrome-stable --version

In [111]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [112]:
!pip show webdriver_manager

Name: webdriver-manager
Version: 4.0.1
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: packaging, python-dotenv, requests
Required-by: 


In [113]:
import pandas as pd
import numpy as np

In [114]:
# Se estiver no Colab, comente o código acima e utilize como alternativa
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.set_page_load_timeout(60)

Exception ignored in: <function ZipFile.__del__ at 0x7c8f8274e200>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [115]:
data = "02/02/2023"
URL_DOL = f"https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-sistema-pregao-ptBR.asp?Data={data}&Mercadoria=DOL"
driver.get(URL_DOL)

In [ ]:
vencimento = driver.find_element(By.ID, "MercadoFut0").text
vencimento

In [ ]:
vencimento = vencimento.split("\n")
vencimento = vencimento[1:]

In [ ]:
vencimento

In [ ]:
tabela_vencimento = pd.DataFrame(vencimento, columns=["VENCTO"])
tabela_vencimento

In [ ]:
n_linhas = tabela_vencimento.shape[0]
n_linhas

In [ ]:
# Captura dos dados do futuro de dólar
dados = driver.find_element(By.ID, "MercadoFut2").text
dados = dados.split()
dados[20:]

In [ ]:

dados = dados[19:] #remoção do cabeçalho
dados

In [ ]:
dados = np.array_split(dados, n_linhas)
dados

In [ ]:
tabela_dados = pd.DataFrame(dados, columns=["PREÇO ABERT", "PREÇO MÍN.", "PREÇO MÁX.", "PREÇO MÉD.", "ÚLT. PREÇO", "AJUSTE", "VAR. PTOS.",	"ÚLT. OF. COMPRA",	"ÚLT. OF. VENDA"])
tabela_dados

In [ ]:
# Consolidar tudo em uma tabela
df_dol = pd.merge(tabela_vencimento, tabela_dados, left_index=True, right_index=True, how="inner")
df_dol

In [102]:
from datetime import date, timedelta

# Definindo o ano para o qual o código será gerado
year = 2023

# Feriados em 2023
holidays = [
    "01/01/2023", "20/02/2023", "21/02/2023", "22/02/2023", "07/04/2023",
    "21/04/2023", "01/05/2023", "08/06/2023", "07/09/2023", "12/10/2023",
    "02/11/2023", "15/11/2023", "25/12/2023", "29/12/2023"
]

# Convertendo strings de feriados para objetos date
holidays = [date.fromisoformat(f"{day[-4:]}-{day[3:5]}-{day[0:2]}") for day in holidays]

# Gerar todas as datas do ano, excluindo sábados, domingos e feriados
start_date = date(year, 1, 1)
end_date = date(year, 12, 31)
all_dates = []

current_date = start_date
while current_date <= end_date:
    if current_date.weekday() < 5 and current_date not in holidays:  # 0-4 são dias da semana (Seg-Sex)
        all_dates.append(current_date.strftime("%d/%m/%Y"))
    current_date += timedelta(days=1)

In [117]:
# Criando um objeto ExcelWriter
with pd.ExcelWriter('dados_dol_2023_por_dia.xlsx', engine='openpyxl') as writer:
    for data in all_dates:
        print(data)
        URL_DOL = f"https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-sistema-pregao-ptBR.asp?Data={data}&Mercadoria=DOL"
        driver.get(URL_DOL)

        vencimento = driver.find_element(By.ID, "MercadoFut0").text.split("\n")[1:]
        tabela_vencimento = pd.DataFrame(vencimento, columns=["VENCTO"])
        n_linhas = tabela_vencimento.shape[0]

        dados = driver.find_element(By.ID, "MercadoFut2").text.split()[19:]
        dados = np.array_split(dados, n_linhas)
        tabela_dados = pd.DataFrame(dados, columns=["PREÇO ABERT", "PREÇO MÍN.", "PREÇO MÁX.", "PREÇO MÉD.", "ÚLT. PREÇO", "AJUSTE", "VAR. PTOS.", "ÚLT. OF. COMPRA", "ÚLT. OF. VENDA"])

        df_dol = pd.merge(tabela_vencimento, tabela_dados, left_index=True, right_index=True, how="inner")

        # Salvando o DataFrame na aba correspondente à data
        df_dol.to_excel(writer, sheet_name=data.replace('/','_'), index=False)

driver.quit()

print("Dados salvos com sucesso em abas separadas por data!")

02/01/2023
03/01/2023
04/01/2023
05/01/2023
06/01/2023
09/01/2023
10/01/2023
11/01/2023
12/01/2023
13/01/2023
16/01/2023
17/01/2023
18/01/2023
19/01/2023
20/01/2023
23/01/2023
24/01/2023
25/01/2023
26/01/2023
27/01/2023
30/01/2023
31/01/2023
01/02/2023
02/02/2023
03/02/2023
06/02/2023
07/02/2023
08/02/2023
09/02/2023
10/02/2023
13/02/2023
14/02/2023
15/02/2023
16/02/2023
17/02/2023
23/02/2023
24/02/2023
27/02/2023
28/02/2023
01/03/2023
02/03/2023
03/03/2023
06/03/2023
07/03/2023
08/03/2023
09/03/2023
10/03/2023
13/03/2023
14/03/2023
15/03/2023
16/03/2023
17/03/2023
20/03/2023
21/03/2023
22/03/2023
23/03/2023
24/03/2023
27/03/2023
28/03/2023
29/03/2023
30/03/2023
31/03/2023
03/04/2023
04/04/2023
05/04/2023
06/04/2023
10/04/2023
11/04/2023
12/04/2023
13/04/2023
14/04/2023
17/04/2023
18/04/2023
19/04/2023
20/04/2023
24/04/2023
25/04/2023
26/04/2023
27/04/2023
28/04/2023
02/05/2023
03/05/2023
04/05/2023
05/05/2023
08/05/2023
09/05/2023
10/05/2023
11/05/2023
12/05/2023
15/05/2023
16/05/2023

TimeoutException: Message: timeout: Timed out receiving message from renderer: 59.037
  (Session info: chrome-headless-shell=124.0.6367.201)
Stacktrace:
#0 0x5bbe37d8ce43 <unknown>
#1 0x5bbe37a7b4e7 <unknown>
#2 0x5bbe37a633bd <unknown>
#3 0x5bbe37a630d3 <unknown>
#4 0x5bbe37a6105c <unknown>
#5 0x5bbe37a616ef <unknown>
#6 0x5bbe37a71c81 <unknown>
#7 0x5bbe37a8726e <unknown>
#8 0x5bbe37a8c87b <unknown>
#9 0x5bbe37a61e32 <unknown>
#10 0x5bbe37a86fe5 <unknown>
#11 0x5bbe37b06f83 <unknown>
#12 0x5bbe37ae81f3 <unknown>
#13 0x5bbe37ab928a <unknown>
#14 0x5bbe37ab9c5e <unknown>
#15 0x5bbe37d5116b <unknown>
#16 0x5bbe37d550bb <unknown>
#17 0x5bbe37d3d281 <unknown>
#18 0x5bbe37d55c22 <unknown>
#19 0x5bbe37d2213f <unknown>
#20 0x5bbe37d7bf98 <unknown>
#21 0x5bbe37d7c170 <unknown>
#22 0x5bbe37d8bf94 <unknown>
#23 0x7c4ae6d3fac3 <unknown>
